In [13]:
Nx = 3; Ny = 3; L = Nx*Ny; N = 3*L 
# t1=-1;L1=0 ;t2=L2=0
# t1=L1=0;t2=L2=-1
t1 = -1;L1 = 0.28;t2 = 0.3;L2 = 0.2;
# t1=t2=-1;L1=L2=0

In [6]:
a1_vec = [2 0]; a2_vec = [1 sqrt(3)]
b1=[0 0];b2=a1_vec/2;b3=a2_vec/2
Basis = [b1,b2,b3]
basis_colors=["red","blue","green"];

In [7]:
using OffsetArrays
coordinates = []; sites = []; atom_dict=Dict{}()
for i1 in 0:Nx-1
    for i2 in 0:Ny-1
        Lp = a1_vec * i1 + a2_vec * i2
        center = a1_vec * i1 + a2_vec * i2 + b1
        shift = (-b2-b3)/sqrt(3)/2
        P1=[center+shift,center+a1_vec+shift,center+a2_vec+shift]
        for (ib, b) in zip(Iterators.countfrom(0), Basis)
            atom_vec = Lp + b
            atom_no = 3 * i1 * Ny + 3 * i2 + ib
            site = [i1,i2,ib]
            sites = push!(sites, site)
            coordinates = push!(coordinates, atom_vec)
            atom_dict[tuple(site)]=atom_vec
        end
    end
end
sites = OffsetArray(sites, 0:26);

In [9]:
NN = [
      [[0,0,1], [0,0,2], [-1,0,1],[0,-1,2],   [-1,0,2], [-1,1,1], [0,-1,1],[1,-1,2]], 
      [[0,0,-1],[0,0,1], [1,0,-1],[1,-1,1],   [0,-1,1], [1,-1,-1],[0,1,-1],[1,0,1]], 
      [[0,0,-1],[0,0,-2],[0,1,-2],[-1,1,-1],  [-1,1,-2],[-1,0,-1],[0,1,-1],[1,0,-2]]
     ]

NN = OffsetArray(NN, 0:2)

hopps = [
         [t1+1im*L1,t1-1im*L1,t1+1im*L1,t1-1im*L1,  t2+1im*L2,t2-1im*L2,t2-1im*L2,t2+1im*L2], 
         [t1-1im*L1,t1+1im*L1,t1-1im*L1,t1+1im*L1,  t2-1im*L2,t2+1im*L2,t2+1im*L2,t2-1im*L2], 
         [t1-1im*L1,t1+1im*L1,t1+1im*L1,t1-1im*L1,  t2-1im*L2,t2+1im*L2,t2+1im*L2,t2-1im*L2]
        ] 

hopps = OffsetArray(hopps, 0:2);

In [12]:
HH = OffsetArray(zeros(Complex{Float64},N,N), 0:N-1, 0:N-1)
for atom_no in 0:N-1
    atom_site=sites[atom_no]
    for (i_delta, delta) in enumerate(NN[atom_site[3]])
        neighbor_site = atom_site+delta
        neighbor_site = OffsetArray(neighbor_site, 0:2)
        neighbor_site[0] = mod(neighbor_site[0],Nx)
        neighbor_site[1] = mod(neighbor_site[1],Ny)
        neighbor_no=3*neighbor_site[0]*Ny+3*neighbor_site[1]+neighbor_site[2]
        HH[neighbor_no,atom_no]=hopps[atom_site[3]][i_delta] 
    end
end

In [11]:
# offset array kullandığımdan linear algenra kütüphanesiyle eigen hesaplayamıyorum mecbur quantumoptics kütüphanesi ile operatöre çevirip hesapladım.
using QuantumOptics
Basis = NLevelBasis(N)
H = SparseOperator(Basis)
for m in 0:N-1
    for n in 0:N-1
        H += HH[m,n] * transition(Basis, m+1, n+1)
    end
end